In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('precision', 2)
from QuantLib import *

## Parameter
----------------

In [ ]:
maturity_date = Date(30, 6, 2018)
start_date = Date(1, 1, 2018)

spot = 1.
strike = 1.
barrier = 1.5

volatility = 0.30
risk_free_rate = 0.
dividend_rate = 0.
day_count = Actual365Fixed()
calendar = China(China.SSE)

## Market Data
-----------------

In [ ]:
calculation_date = start_date
Settings.instance().evaluationDate = calculation_date

spot_handle = RelinkableQuoteHandle(SimpleQuote(spot))
vol_handle =  RelinkableQuoteHandle(SimpleQuote(volatility))

flat_ts = YieldTermStructureHandle(FlatForward(calculation_date, risk_free_rate, day_count))
dividend_yield = YieldTermStructureHandle(FlatForward(calculation_date, dividend_rate, day_count))
flat_vol_ts = BlackVolTermStructureHandle(BlackConstantVol(calculation_date, calendar, vol_handle, day_count))

bsm_process = BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)

In [ ]:
payoff = PlainVanillaPayoff(Option.Call, strike)
exercise = EuropeanExercise(maturity_date)

option = BarrierOption(Barrier.UpOut, barrier=barrier, rebate=0., payoff=payoff, exercise=exercise)

## Analytic Method
------------------

In [ ]:
%%time
engine = AnalyticBarrierEngine(bsm_process)
option.setPricingEngine(engine)

bsm_price = option.NPV()
print("BSM european theoreticl price is {0:.4f}".format(bsm_price))

## FDM method
-------------

In [ ]:
%%time
engine = FdBlackScholesBarrierEngine(bsm_process, 100, 100)
option.setPricingEngine(engine)

bsm_price = option.NPV()
bsm_delta = option.delta()
print("FDM european price is {0:.4f}".format(bsm_price))
print("FDM european delta is {0:.4f}".format(bsm_delta))

## MC Method
-------------------

In [ ]:
engine = MCBarrierEngine(bsm_process, 'ld', 100, brownianBridge=True, antitheticVariate=True, requiredSamples=5000)
option.setPricingEngine(engine)

bsm_price = option.NPV()
print("MC european price is {0:.4f}".format(bsm_price))